SolverWorld's [Computing speed for kore harvesting routes](https://www.kaggle.com/code/solverworld/computing-speed-for-kore-harvesting-routes/notebook) notebook shows how important it can be to tune calculations for speed in a time-limited challenge like this. I thought it'd be fun to show the value of checking the various ways you can do things in numpy since you can often make savings by trying out a few approaches.

One thing many must all be doing somewhere is multiplying an array of kore by the growth rate of 1.02. Let's see how we can approach it and refine it. This example isn't particularly useful, but shows how there are many avenues available for tuning.

In [ ]:
import numpy as np

In [ ]:
# randomly distributed array
board_kore = abs(np.random.uniform(0,1, size = (21,21)))*500
board_kore.dtype

Time a simple multiplication

In [ ]:
%%timeit
1.02 * board_kore

Let's give numpy a helping hand

In [ ]:
a = np.zeros((21,21), dtype=float)
a[:,:]=1.02

Does it help?

In [ ]:
%%timeit
a * board_kore

Now let's more directly tell numpy what we're trying to do. Does it make it faster?

In [ ]:
%%timeit
np.multiply(a, board_kore)

Let's pre-allocate the output array.

In [ ]:
out = np.empty_like(a)

In [ ]:
%%timeit
np.multiply(a, board_kore, out)

It seems to help!

Next, alias the method to save the np lookup each time.

In [ ]:
m = np.multiply

In [ ]:
%%timeit
m(a, board_kore, out)

Around a 40-50% saving.

Let's check the results of each method are equivalent.

In [ ]:
t1 = 1.02 * board_kore
t2 = a * board_kore
t3 = np.multiply(a, board_kore)
t4 = np.multiply(a, board_kore, out)
t5 = m(a, board_kore, out)

np.allclose(t1,t2), np.allclose(t2,t3), np.allclose(t3,t4), np.allclose(t4,t5)

As a second example, we can take the excellent [Halite Simultor 10x performance boost](https://www.kaggle.com/code/elvenmonk/halite-numpy-simulator-10x-performance-boost) notebook and make part of it 5x faster.

It uses np.roll() to move positions, but this is quite slow with single positions shifts of small matrices. We can speed it up as such.


In [ ]:
%%timeit
np.roll(board_kore, -1, axis=0)


In [ ]:
%%timeit
np.concatenate([board_kore[1:],board_kore[:1]])

Check equivalence

In [ ]:
t1 = np.roll(board_kore, -1, axis = 0)
t2 = np.concatenate([board_kore[1:],board_kore[:1]])
np.allclose(t1,t2)